TODO

Summarize with summa and pretty it with openai

In [4]:
import tiktoken
from pathlib import Path

def get_encoding():
    # Try to match ChatGPT-like models, fallback to generic encoding
    try:
        return tiktoken.encoding_for_model("gpt-4o-mini")
    except KeyError:
        # cl100k_base works for GPT-4/4o/3.5 style models
        return tiktoken.get_encoding("cl100k_base")

encoding = get_encoding()

def count_tokens(text: str) -> int:
    tokens = encoding.encode(text)
    return len(tokens)

In [5]:

# read the text directly from sample_text.txt
sample_path = Path("sample_text.txt")
sample_text = sample_path.read_text(encoding="utf-8")

print("Token count:", count_tokens(sample_text))

Token count: 8777


In [6]:
from summa.summarizer import summarize

# -----------------------------------------------------
# Generate summary using a ratio of the text length
# ratio=0.1 → summary = ~10% of original length
# You can increase to 0.2 or 0.3 if needed
# -----------------------------------------------------
summary = summarize(
    sample_text,
    ratio=0.1,          # keep top 10% of sentences
    split=False         # return a single string
)

# -----------------------------------------------------
# If result is empty (sometimes very large documents need word-based)
# switch to 'words=' parameter, e.g. 300 or 500 words
# -----------------------------------------------------
if not summary.strip():
    summary = summarize(
        sample_text,
        words=300,       # fallback summary length
        split=False
    )

print("\n================ SUMMARY ================\n")
print(summary)
print("\n=========================================\n")



================ SUMMARY ================

[μουσική] Απ [μουσική] ότι είδα η αστυνομία αν σε σταματήσει σε έλεγχο δεν έχει τρόπο να ανακαλύψει αν έχεις κάνει χρήση ουσιών πέρα από το alcκοτε test, αλλά απ ότι διάβασα σχεδιάζουνε πλέον στον έλεγκο που θα κάνουν οι αστυνομικοί να έχουν τη δυνατότητα εκείνη τη στιγμή να δουν για είσαι μαστορωμένος.
Ο 29χρονος οδηγός δευτερόλεπτα πριν από το φρικτό τροχαίο δυστούχημα έχει καταγραφεί σε βίντεο ντοκουμέντο να κινείται με ηλιγκιώδη ταχύτητα σε έναν δρόμο που το όριο ταχύτητας είναι 30 km.
Αυτός έλεγε ότι με πονάει η ταυτή μου και να δω τι ζύμιες έχει το αμάξι και κοίταγε το αμάξι του.
Όπως έγινε γνωστό, ο 24χρονος που σκοτώθηκε είχε μόλις έρθει από το Βέλγιο όπου ζούσε για να δώσει το παρόν στην ορκομοσία της κοπέλας του και όλο χαρά μας είπε ότι ήρθε ο Σωτήρης μου στην Ελλάδα.
Σωτήρη τον λέγανε έτσι ήρθε απ το Βέλγιο στην Ελλάδα να δει την οργομοσία της κοπέλας του και του συνέβει αυτό.
Η αστυνομία τι κάνει απ τη στιγμή να πούμε που έγινε τ

In [ ]:
def chunk_file_to_txt_files(
    input_path: str,
    output_dir: str = "chunks",
    max_tokens: int = 2000
):
    # Prepare paths
    input_path = Path(input_path)
    out_dir = Path(output_dir)
    out_dir.mkdir(exist_ok=True)

    # Read the input text
    text = input_path.read_text(encoding="utf-8")

    # Tokenize
    token_ids = encoding.encode(text)

    # Split into chunks
    chunks = []
    for i in range(0, len(token_ids), max_tokens):
        chunk_ids = token_ids[i : i + max_tokens]
        chunk_text = encoding.decode(chunk_ids)
        chunks.append(chunk_text)

    # Write each chunk into its own .txt file
    for idx, chunk in enumerate(chunks, start=1):
        output_file = out_dir / f"chunk_{idx}.txt"
        output_file.write_text(chunk, encoding="utf-8")

    print(f"Created {len(chunks)} chunk files in '{output_dir}' folder.")
    return chunks

In [ ]:
chunks = chunk_file_to_txt_files(
    input_path="sample_text.txt",
    output_dir="chunks_output",
    max_tokens=2000   # you can tune this
)